### Machine Learning / Asunciones - Pair Programming

El objetivo de este pairprgramming es que evaluéis si vuestro set de datos cumple todas las asunciones que se deben cumplir para hacer una regresión lineal. Recordamos que estas asunciones son:
- Normalidad (ya la deberíais haber evaluado).
- Homogeneidad de varianzas.
- Independencia de las variables.

Cada asunción la deberéis testar de forma visual y analítica.

In [72]:
import pandas as pd
import numpy as np
from scipy.stats import kstest

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (10,8)

import warnings
warnings.filterwarnings('ignore')

In [73]:
df = pd.read_csv("data/adult.data_limpio.csv", index_col = 0)

- Asunción de normalidad:

In [74]:
df.shape

(32560, 11)

In [75]:
df.head()

,state-gov,bachelors,education_num,marital_status,ocupation,relationship,ethnicity,gender,hours_week,country,salary
39,,,,,,,,,,,
50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K
37,Private,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K


Observamos que nuestro DataFrame tiene más de 5 mil líneas, por lo tanto vamos a usar el test de Kolmogórov-Smirnov.

**Convertir la columna salary <=50K en 0 y >=50k en 1.**

In [76]:
df["salary"].unique()

array([' <=50K', ' >50K'], dtype=object)

In [97]:
def salario(col):
        for i in col:
            print(i)
            if i == " <=50K":
                return 0
            elif i == " >50K":
                return 1
        else:
            print("No funciona reinas")

        return col

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 11)

In [95]:
salario("salary")

s
a
l
a
r
y
No funciona reinas


'salary'

In [93]:
df.head()

,state-gov,bachelors,education_num,marital_status,ocupation,relationship,ethnicity,gender,hours_week,country,salary
39,,,,,,,,,,,
50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K
37,Private,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K


In [80]:
kstest(df["salary"], 'norm')

TypeError: unsupported operand type(s) for -: 'str' and 'int'